In [34]:
# Code adapted from the work of Richard Kuo shared on Kaggle
# The Python code has been published at: https://www.kaggle.com/rkuo2000/financial-statement-analysis
# import libraries
from datetime import datetime
import lxml
from lxml import html
import requests
import numpy as np
import pandas as pd

In [35]:
# input a stock symbol
symbolMRU = 'MRU.TO'

In [36]:
# Yahoo Finance links
urlMRU_bs = 'https://finance.yahoo.com/quote/' + symbolMRU + '/balance-sheet?p=' + symbolMRU
urlMRU_is = 'https://finance.yahoo.com/quote/' + symbolMRU + '/financials?p=' + symbolMRU
urlMRU_cf = 'https://finance.yahoo.com/quote/' + symbolMRU + '/cash-flow?p='+ symbolMRU

In [37]:
# Set up the request headers that we're going to use, to simulate a request by the Chrome browser. 
# Simulating a request from a browser is generally good practice when building a scraper
headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9',
    'Cache-Control': 'max-age=0',
    'Pragma': 'no-cache',
    'Referrer': 'https://google.com',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.120 Safari/537.36'
}

In [38]:
def get_table(url):
    # Fetch the page that we're going to parse, using the request headers defined above
    page = requests.get(url, headers)

    # Parse the page with LXML, so that we can start doing some XPATH queries
    # to extract the data that we want
    tree = html.fromstring(page.content)

    # Smoke test that we fetched the page by fetching and displaying the H1 element
    tree.xpath("//h1/text()")
    table_rows = tree.xpath("//div[contains(@class, 'D(tbr)')]")

    # Ensure that some table rows are found; if none are found, then it's possible
    # that Yahoo Finance has changed their page layout, or have detected
    # that you're scraping the page.
    assert len(table_rows) > 0

    parsed_rows = []

    for table_row in table_rows:
        parsed_row = []
        el = table_row.xpath("./div")

        none_count = 0

        for rs in el:
            try:
                (text,) = rs.xpath('.//span/text()[1]')
                parsed_row.append(text)
            except ValueError:
                parsed_row.append(np.NaN)
                none_count += 1

        if (none_count < 4):
            parsed_rows.append(parsed_row)

    df = pd.DataFrame(parsed_rows)
    df_org = df
    
    df = pd.DataFrame(parsed_rows)
    df = df.set_index(0) # Set the index to the first column: 'Period Ending'.
    df = df.transpose() # Transpose the DataFrame, so that our header contains the account names

    # Rename the "Breakdown" column to "Date"
    cols = list(df.columns)
    cols[0] = 'Date'
    df = df.set_axis(cols, axis='columns', inplace=False)
    df_rot = df
    return df_org, df_rot

# Metro


In [39]:
# get Balance Sheet 
urlBSMRU = 'https://raw.githubusercontent.com/hadrienpierre/IND8122-CS1/main/Annual_BalanceSheet_MRU.csv'
BSMRU_orginal= pd.read_csv (urlBSMRU, index_col=0, decimal='.')
BSMRU_transpose = BSMRU_orginal.transpose() 

# get Income Statement 
urlISMRU = 'https://raw.githubusercontent.com/hadrienpierre/IND8122-CS1/main/Annual_IncomeStatement_MRU.csv'
ISMRU_orginal = pd.read_csv (urlISMRU, index_col=0, decimal='.')
ISMRU_transpose = ISMRU_orginal.transpose()

# get Cash Flow
CFMRU_orginal, CFMRU_transpose = get_table(urlMRU_cf)

In [40]:
BSMRU_orginal

,9/30/2019,9/30/2018,9/30/2017,9/30/2016,9/30/2015
Date fin exercice,,,,,
Total de l'actif,11073.9,10922.2,6050.7,5606.1,5387.1
Actifs courants,2088.3,1916.8,1356.3,1193,1172.9
Trésorerie équivalents de trésorerie et placements à court terme,27340,22690,14890,2750,2150
Trésorerie et équivalents de trésorerie,27340,22690,14890,2750,2150
En espèces,—,—,—,—,—
Équivalents de trésorerie,—,—,—,—,—
Placements à court terme,—,—,—,—,—
Débiteurs,65570,55870,33180,31830,30370
Comptes clients,61120,53810,31370,30640,29060


In [41]:
ISMRU_orginal
ISMRU_transpose

Date fin exercice,Total des produits,Produits d'exploitation,Taxes d’accise,Coût des produits,Profit brut,Charges d’exploitation,Frais de vente et charges générales et administratives,Charges générales et administratives,Salaires,Loyer et redevances d’atterrissage,...,Coût des produits après rapprochement,Amortissement des immobilisations corporelles après rapprochement,Bénéfice net découlant des activités poursuivies après déduction pour participation minoritaire,Total des éléments exceptionnels à l’exclusion du goodwill,Total des éléments exceptionnels,Bénéfice de base par action normalisé,Bénéfice dilué par action normalisé,BAIIA normalisé,Taux d’impôt pour les calculs,Effet fiscal des éléments exceptionnels
9/30/2019,16767.5,16767.5,—,13438.8,3328.7,226360,149560,149560,96640,52920,...,—,—,—,—,—,—,—,—,—,—
9/30/2018,14383.4,14383.4,—,11556.5,2826.9,200650,133870,133870,86290,47580,...,—,—,—,—,—,—,—,—,—,—
9/30/2017,13175.3,13175.3,—,10579.6,2595.7,182350,123320,123320,79180,44140,...,—,—,—,—,—,—,—,—,—,—
9/30/2016,12787.9,12787.9,—,10271.1,2516.8,176830,119630,119630,77560,42070,...,—,—,—,—,—,—,—,—,—,—
9/30/2015,12223.8,12223.8,—,9813.5,2410.3,172950,116610,116610,75650,40960,...,—,—,—,—,—,—,—,—,—,—


In [42]:
CFMRU_orginal

,0,1,2,3,4
0,Breakdown,ttm,9/30/2019,9/30/2018,9/30/2017
1,Operating Cash Flow,"1,184,500","687,700","750,400","696,200"
2,Investing Cash Flow,"-408,300","-308,500","-1,677,500","-333,000"
3,Financing Cash Flow,"-672,900","-332,700","1,005,100","-241,800"
4,End Cash Position,"366,600","273,400","226,900","148,900"
5,Capital Expenditure,"-460,000","-396,300","-317,400","-368,900"
6,Issuance of Capital Stock,"14,600","24,000","8,800","10,700"
7,Issuance of Debt,"413,400","46,600","2,168,800","737,700"
8,Repayment of Debt,"-633,100","-54,000","-996,200","-537,300"
9,Repurchase of Capital Stock,"-214,600","-151,500","-10,200","-309,500"


In [43]:
# Balance Sheet transpose table
BSMRU_transpose

Date fin exercice,Total de l'actif,Actifs courants,Trésorerie équivalents de trésorerie et placements à court terme,Trésorerie et équivalents de trésorerie,En espèces,Équivalents de trésorerie,Placements à court terme,Débiteurs,Comptes clients,Comptes clients bruts,...,Actifs corporels nets,Fonds de roulement,Capital investi,Valeur comptable des immobilisations corporelles,Total de la dette,Dette nette,Actions émises,Nombre d’actions ordinaires,Nombre d’actions privilégiées,Nombre d’actions propres
9/30/2019,11073.9,2088.3,27340,27340,—,—,—,65570,61120,—,...,—,—,—,—,—,—,—,—,—,—
9/30/2018,10922.2,1916.8,22690,22690,—,—,—,55870,53810,—,...,—,—,—,—,—,—,—,—,—,—
9/30/2017,6050.7,1356.3,14890,14890,—,—,—,33180,31370,—,...,—,—,—,—,—,—,—,—,—,—
9/30/2016,5606.1,1193,2750,2750,—,—,—,31830,30640,—,...,—,—,—,—,—,—,—,—,—,—
9/30/2015,5387.1,1172.9,2150,2150,—,—,—,30370,29060,—,...,—,—,—,—,—,—,—,—,—,—


### Ratio du Fond de Roulement = Actif Courant / Passif Courant

In [44]:
current_assets = BSMRU_transpose["Actifs courants"].str.replace(',', '').astype(float)
current_liabilities = BSMRU_transpose["Passifs courants"].str.replace(',', '').astype(float)
rfdr = current_assets / current_liabilities
BSMRU_analysis = pd.DataFrame(rfdr)
BSMRU_analysis.columns = ['Ratio Fond de Roulement']
BSMRU_analysis

,Ratio Fond de Roulement
9/30/2019,1.112218
9/30/2018,1.172570
9/30/2017,1.054748
9/30/2016,1.117564
9/30/2015,1.102557


## Levier Financier

### Endettement Total

In [45]:
capitaux_propres = BSMRU_transpose["Capitaux propres"].str.replace(',', '').astype(float)
total_assets = BSMRU_transpose["Total de l'actif"].str.replace(',', '').astype(float)
BSMRU_analysis['Endettement Total'] = (total_assets - capitaux_propres) / total_assets
BSMRU_analysis

,Ratio Fond de Roulement,Endettement Total
9/30/2019,1.112218,0.462231
9/30/2018,1.172570,0.483364
9/30/2017,1.054748,0.518882
9/30/2016,1.117564,0.521842
9/30/2015,1.102557,0.509309


### Dette/Fonds Propres

In [46]:
total_liabilities = BSMRU_transpose["Total du passif"].str.replace(',', '').astype(float)
BSMRU_analysis['Dettes-Fonds Propres'] = total_liabilities / capitaux_propres
BSMRU_analysis

,Ratio Fond de Roulement,Endettement Total,Dettes-Fonds Propres
9/30/2019,1.112218,0.462231,0.857284
9/30/2018,1.172570,0.483364,0.933260
9/30/2017,1.054748,0.518882,1.074096
9/30/2016,1.117564,0.521842,1.086660
9/30/2015,1.102557,0.509309,1.032723


### Endettement à long terme
Ratio d'endettement à long terme = Passif à long terme / (Passif à long terme + Fonds Propres)

In [47]:
longterm_liabilities = BSMRU_transpose["Total des passifs non courants"].str.replace(',', '').astype(float)
BSMRU_analysis['Endettement Long Terme'] = longterm_liabilities / (longterm_liabilities + capitaux_propres)
BSMRU_analysis

,Ratio Fond de Roulement,Endettement Total,Dettes-Fonds Propres,Endettement Long Terme
9/30/2019,1.112218,0.462231,0.857284,0.351490
9/30/2018,1.172570,0.483364,0.933260,0.391566
9/30/2017,1.054748,0.518882,1.074096,0.387395
9/30/2016,1.117564,0.521842,1.086660,0.407733
9/30/2015,1.102557,0.509309,1.032723,0.386611


## Utilisation de l'actif

### Rotation des Stocks
Coûts des marchandises vendues / Stocks

In [48]:
cogs = ISMRU_transpose["Coût des produits"].str.replace(',', '').astype(float)
inventory = BSMRU_transpose["Stocks"].str.replace(',', '').astype(float)
BSMRU_analysis['Rotation des stocks'] = cogs / inventory
BSMRU_analysis

,Ratio Fond de Roulement,Endettement Total,Dettes-Fonds Propres,Endettement Long Terme,Rotation des stocks
9/30/2019,1.112218,0.462231,0.857284,0.351490,11.934991
9/30/2018,1.172570,0.483364,0.933260,0.391566,10.514512
9/30/2017,1.054748,0.518882,1.074096,0.387395,12.350689
9/30/2016,1.117564,0.521842,1.086660,0.407733,12.412205
9/30/2015,1.102557,0.509309,1.032723,0.386611,11.906697


### Rotation de l'actif
Chiffre d'affaire / Total de l'actif

In [49]:
revenues = ISMRU_transpose["Total des produits"].str.replace(',', '').astype(float)
BSMRU_analysis["Rotation de l'actif"] = revenues / total_assets
BSMRU_analysis

,Ratio Fond de Roulement,Endettement Total,Dettes-Fonds Propres,Endettement Long Terme,Rotation des stocks,Rotation de l'actif
9/30/2019,1.112218,0.462231,0.857284,0.351490,11.934991,1.514146
9/30/2018,1.172570,0.483364,0.933260,0.391566,10.514512,1.316896
9/30/2017,1.054748,0.518882,1.074096,0.387395,12.350689,2.177484
9/30/2016,1.117564,0.521842,1.086660,0.407733,12.412205,2.281069
9/30/2015,1.102557,0.509309,1.032723,0.386611,11.906697,2.269087


In [50]:
## Rentablité

### Marge Bénéficiaire
Bénéfice net / Chiffre d'affaire


In [51]:
profits = ISMRU_transpose["Bénéfice net"].str.replace(',', '').astype(float)
BSMRU_analysis["Marge Bénéficiaire"] = profits / revenues
BSMRU_analysis

,Ratio Fond de Roulement,Endettement Total,Dettes-Fonds Propres,Endettement Long Terme,Rotation des stocks,Rotation de l'actif,Marge Bénéficiaire
9/30/2019,1.112218,0.462231,0.857284,0.351490,11.934991,1.514146,0.042439
9/30/2018,1.172570,0.483364,0.933260,0.391566,10.514512,1.316896,0.119339
9/30/2017,1.054748,0.518882,1.074096,0.387395,12.350689,2.177484,0.044910
9/30/2016,1.117564,0.521842,1.086660,0.407733,12.412205,2.281069,0.044691
9/30/2015,1.102557,0.509309,1.032723,0.386611,11.906697,2.269087,0.041403


### Rendement de l'actif
Bénéfice net/Total de l'actif

In [52]:
BSMRU_analysis["Rendement de l'actif"] = profits / total_assets
BSMRU_analysis

,Ratio Fond de Roulement,Endettement Total,Dettes-Fonds Propres,Endettement Long Terme,Rotation des stocks,Rotation de l'actif,Marge Bénéficiaire,Rendement de l'actif
9/30/2019,1.112218,0.462231,0.857284,0.351490,11.934991,1.514146,0.042439,0.064259
9/30/2018,1.172570,0.483364,0.933260,0.391566,10.514512,1.316896,0.119339,0.157157
9/30/2017,1.054748,0.518882,1.074096,0.387395,12.350689,2.177484,0.044910,0.097790
9/30/2016,1.117564,0.521842,1.086660,0.407733,12.412205,2.281069,0.044691,0.101943
9/30/2015,1.102557,0.509309,1.032723,0.386611,11.906697,2.269087,0.041403,0.093947


### Rendement des fonds propres
Bénéfice Net / total des fonds propres


In [53]:
BSMRU_analysis["Rendement des fonds propres"] = profits / capitaux_propres
BSMRU_analysis

,Ratio Fond de Roulement,Endettement Total,Dettes-Fonds Propres,Endettement Long Terme,Rotation des stocks,Rotation de l'actif,Marge Bénéficiaire,Rendement de l'actif,Rendement des fonds propres
9/30/2019,1.112218,0.462231,0.857284,0.351490,11.934991,1.514146,0.042439,0.064259,0.119492
9/30/2018,1.172570,0.483364,0.933260,0.391566,10.514512,1.316896,0.119339,0.157157,0.304193
9/30/2017,1.054748,0.518882,1.074096,0.387395,12.350689,2.177484,0.044910,0.097790,0.203257
9/30/2016,1.117564,0.521842,1.086660,0.407733,12.412205,2.281069,0.044691,0.101943,0.213199
9/30/2015,1.102557,0.509309,1.032723,0.386611,11.906697,2.269087,0.041403,0.093947,0.191458


In [54]:
#Croissance des ventes

In [55]:
ISMRU_transpose["Total des produits"]

9/30/2019    16767.5
9/30/2018    14383.4
9/30/2017    13175.3
9/30/2016    12787.9
9/30/2015    12223.8
Name: Total des produits, dtype: object

In [56]:
Croissance_Ventes =  -1 + ISMRU_transpose["Total des produits"].str.replace(',', '').astype(float).div(ISMRU_transpose["Total des produits"].str.replace(',', '').astype(float).shift(-1))
ProFormaRatio = pd.DataFrame (Croissance_Ventes)
ProFormaRatio.columns = ['Croissance Ventes']
ProFormaRatio

,Croissance Ventes
9/30/2019,0.165754
9/30/2018,0.091694
9/30/2017,0.030294
9/30/2016,0.046148
9/30/2015,NaN


In [57]:
test_array = [2, 4, 6, 8, 10, 5]
test = pd.DataFrame(test_array)
test.columns = ['Ville']
test
habitant = [3, 12, 16, 24, 30, 15]
test['Habitant'] = habitant
test
test['Sanity'] = test['Habitant'].div(test['Ville'])
test
test['Croissance'] = test['Habitant'].div(test['Habitant'].shift(-1))
test


,Ville,Habitant,Sanity,Croissance
0,2,3,1.500000,0.250000
1,4,12,3.000000,0.750000
2,6,16,2.666667,0.666667
3,8,24,3.000000,0.800000
4,10,30,3.000000,2.000000
5,5,15,3.000000,NaN


In [58]:
BSMRU_analysis = BSMRU_analysis.transpose()
BSMRU_analysis["Moyenne 5 ans"] = BSMRU_analysis.mean(axis=1)
BSMRU_analysis

,9/30/2019,9/30/2018,9/30/2017,9/30/2016,9/30/2015,Moyenne 5 ans
Ratio Fond de Roulement,1.112218,1.172570,1.054748,1.117564,1.102557,1.111931
Endettement Total,0.462231,0.483364,0.518882,0.521842,0.509309,0.499126
Dettes-Fonds Propres,0.857284,0.933260,1.074096,1.086660,1.032723,0.996805
Endettement Long Terme,0.351490,0.391566,0.387395,0.407733,0.386611,0.384959
Rotation des stocks,11.934991,10.514512,12.350689,12.412205,11.906697,11.823819
Rotation de l'actif,1.514146,1.316896,2.177484,2.281069,2.269087,1.911736
Marge Bénéficiaire,0.042439,0.119339,0.044910,0.044691,0.041403,0.058556
Rendement de l'actif,0.064259,0.157157,0.097790,0.101943,0.093947,0.103019
Rendement des fonds propres,0.119492,0.304193,0.203257,0.213199,0.191458,0.206320


In [59]:

# input a stock symbol
symbol = 'EMP-A.TO'
# Yahoo Finance links
url_bs = 'https://finance.yahoo.com/quote/' + symbol + '/balance-sheet?p=' + symbol
url_is = 'https://finance.yahoo.com/quote/' + symbol + '/financials?p=' + symbol
url_cf = 'https://finance.yahoo.com/quote/' + symbol + '/cash-flow?p='+ symbol
# get Balance Sheet 
BS_orginal, BS_transpose = get_table(url_bs)

# get Income Statement 
IS_orginal, IS_transpose = get_table(url_is)

# get Cash Flow
CF_orginal, CF_transpose = get_table(url_cf)

BS_analysis = pd.DataFrame(BS_transpose['Date']) # copy columns of dataframe
BS_analysis
BS_orginal

,0,1,2,3,4
0,Breakdown,4/30/2020,4/30/2019,4/30/2018,4/30/2017
1,Total Assets,"14,632,900","9,602,400","8,662,000","8,695,500"
2,Total Liabilities Net Minority Interest,"10,619,000","5,519,400","4,892,200","4,992,800"
3,Total Equity Gross Minority Interest,"4,013,900","4,083,000","3,769,800","3,702,700"
4,Total Capitalization,"5,029,800","5,987,700","4,842,300","5,381,000"
5,Common Stock Equity,"3,924,600","4,003,300","3,702,800","3,644,200"
6,Capital Lease Obligations,"5,266,200",NaN,NaN,NaN
7,Net Tangible Assets,"1,382,100","1,369,800","1,858,900","1,760,300"
8,Working Capital,"-804,500","-41,800","-466,300","-324,200"
9,Invested Capital,"5,599,800","6,024,200","5,369,700","5,515,000"


In [60]:
# get Balance Sheet 
urlBSMRU = 'https://raw.githubusercontent.com/hadrienpierre/IND8122-CS1/main/Annual_BalanceSheet_MRU.csv'
BSMRU_orginal= pd.read_csv (urlBSMRU, index_col=0, decimal='.')
BSMRU_transpose = BSMRU_orginal.transpose() 

# get Income Statement 
urlISMRU = 'https://raw.githubusercontent.com/hadrienpierre/IND8122-CS1/main/Annual_IncomeStatement_MRU.csv'
ISMRU_orginal = pd.read_csv (urlISMRU, index_col=0, decimal='.')
ISMRU_transpose = ISMRU_orginal.transpose()

# get Cash Flow
CFMRU_orginal, CFMRU_transpose = get_table(urlMRU_cf)

#Ratio du Fond de Roulement = Actif Courant / Passif Courant

current_assets = BSMRU_transpose["Actifs courants"].str.replace(',', ' ').astype(float)
current_liabilities = BSMRU_transpose["Passifs courants"].str.replace(',', ' ').astype(float)
rfdr = current_assets / current_liabilities
BSMRU_analysis = pd.DataFrame(rfdr)
BSMRU_analysis.columns = ['Ratio Fond de Roulement']

#Levier Financier
# Endettement Total
capitaux_propres = BSMRU_transpose["Capitaux propres"].str.replace(',', ' ').astype(float)
total_assets = BSMRU_transpose["Total de l'actif"].str.replace(',', ' ').astype(float)
BSMRU_analysis['Endettement Total'] = (total_assets - capitaux_propres) / total_assets
BSMRU_analysis

# Dette Fonds Propres
total_liabilities = BSMRU_transpose["Total du passif"].str.replace(',', ' ').astype(float)
BSMRU_analysis['Dettes-Fonds Propres'] = total_liabilities / capitaux_propres
BSMRU_analysis

#Endettement long terme
longterm_liabilities = BSMRU_transpose["Total des passifs non courants"].str.replace(',', ' ').astype(float)
BSMRU_analysis['Endettement Long Terme'] = longterm_liabilities / (longterm_liabilities + capitaux_propres)
BSMRU_analysis

#Rotation des Stocks
cogs = ISMRU_transpose["Coût des produits"].str.replace(',', ' ').astype(float)
inventory = BSMRU_transpose["Stocks"].str.replace(',', ' ').astype(float)
BSMRU_analysis['Rotation des stocks'] = cogs / inventory
BSMRU_analysis

#Rotation de l'actif
revenues = ISMRU_transpose["Total des produits"].str.replace(',', ' ').astype(float)
BSMRU_analysis["Rotation de l'actif"] = revenues / total_assets
BSMRU_analysis

## Rentablité
#Marge Bénéficiaire
profits = ISMRU_transpose["Bénéfice net"].str.replace(',', ' ').astype(float)
BSMRU_analysis["Marge Bénéficiaire"] = profits / revenues
BSMRU_analysis

# Rendement de l'actif
BSMRU_analysis["Rendement de l'actif"] = profits / total_assets
BSMRU_analysis

# Rendement des fonds propres
BSMRU_analysis["Rendement des fonds propres"] = profits / capitaux_propres
BSMRU_analysis

BSMRU_analysis = BSMRU_analysis.transpose()
BSMRU_analysis["Moyenne 5 ans"] = BSMRU_analysis.mean(axis=1)
BSMRU_analysis


,9/30/2019,9/30/2018,9/30/2017,9/30/2016,9/30/2015,Moyenne 5 ans
Ratio Fond de Roulement,1.112218,1.172570,1.054748,1.117564,1.102557,1.111931
Endettement Total,0.462231,0.483364,0.518882,0.521842,0.509309,0.499126
Dettes-Fonds Propres,0.857284,0.933260,1.074096,1.086660,1.032723,0.996805
Endettement Long Terme,0.351490,0.391566,0.387395,0.407733,0.386611,0.384959
Rotation des stocks,11.934991,10.514512,12.350689,12.412205,11.906697,11.823819
Rotation de l'actif,1.514146,1.316896,2.177484,2.281069,2.269087,1.911736
Marge Bénéficiaire,0.042439,0.119339,0.044910,0.044691,0.041403,0.058556
Rendement de l'actif,0.064259,0.157157,0.097790,0.101943,0.093947,0.103019
Rendement des fonds propres,0.119492,0.304193,0.203257,0.213199,0.191458,0.206320


In [66]:
BSMRU_analysis.to_csv (r'C:\Users\hadri\OneDrive\Documents\Cours\A20\IND8122\export_BSMRU.csv', index = True, header=True)

In [61]:
# get Balance Sheet 
urlBSL = 'https://raw.githubusercontent.com/hadrienpierre/IND8122-CS1/main/Annual_BalanceSheet_L.csv'
BSL_orginal= pd.read_csv (urlBSL, index_col=0, decimal='.', thousands = ' ')
BSL_transpose = BSL_orginal.transpose() 

# get Income Statement 
urlISL = 'https://raw.githubusercontent.com/hadrienpierre/IND8122-CS1/main/Annual_IncomeStatement_L.csv'
ISL_orginal = pd.read_csv (urlISL, index_col=0, decimal='.', thousands = ' ')
ISL_transpose = ISL_orginal.transpose()

# get Cash Flow
#CFL_orginal, CFL_transpose = get_table(urlL_cf)

#Ratio du Fond de Roulement = Actif Courant / Passif Courant

current_assets = BSL_transpose["Actifs courants"].str.replace(',', ' ').astype(float)
current_liabilities = BSL_transpose["Passifs courants"].str.replace(',', ' ').astype(float)
rfdr = current_assets / current_liabilities
BSL_analysis = pd.DataFrame(rfdr)
BSL_analysis.columns = ['Ratio Fond de Roulement']

#Levier Financier
# Endettement Total
capitaux_propres = BSL_transpose["Capitaux propres"].str.replace(',', ' ').astype(float)
total_assets = BSL_transpose["Total de l'actif"].str.replace(',', ' ').astype(float)
BSL_analysis['Endettement Total'] = (total_assets - capitaux_propres) / total_assets
BSL_analysis

# Dette Fonds Propres
total_liabilities = BSL_transpose["Total du passif"].str.replace(',', ' ').astype(float)
BSL_analysis['Dettes-Fonds Propres'] = total_liabilities / capitaux_propres
BSL_analysis

#Endettement long terme
longterm_liabilities = BSL_transpose["Total des passifs non courants"].str.replace(',', ' ').astype(float)
BSL_analysis['Endettement Long Terme'] = longterm_liabilities / (longterm_liabilities + capitaux_propres)
BSL_analysis

#Rotation des Stocks
cogs = ISL_transpose["Coût des produits"].str.replace(',', ' ').astype(float)
inventory = BSL_transpose["Stocks"].str.replace(',', '').astype(float)
BSL_analysis['Rotation des stocks'] = cogs / inventory
BSL_analysis

#Rotation de l'actif
revenues = ISL_transpose["Total des produits"].str.replace(',', ' ').astype(float)
BSL_analysis["Rotation de l'actif"] = revenues / total_assets
BSL_analysis

## Rentablité
#Marge Bénéficiaire
profits = ISL_transpose["Bénéfice net"].str.replace(',', ' ').astype(float)
BSL_analysis["Marge Bénéficiaire"] = profits / revenues
BSL_analysis

# Rendement de l'actif
BSL_analysis["Rendement de l'actif"] = profits / total_assets
BSL_analysis

# Rendement des fonds propres
BSL_analysis["Rendement des fonds propres"] = profits / capitaux_propres
BSL_analysis

BSL_analysis = BSL_analysis.transpose()
BSL_analysis["Moyenne 5 ans"] = BSL_analysis.mean(axis=1)
BSL_analysis


,déc. 19,déc. 18,déc. 17,déc. 16,déc. 15,Moyenne 5 ans
Ratio Fond de Roulement,1.225751,1.336972,1.310844,1.469893,1.364580,1.341608
Endettement Total,0.690600,0.598083,0.627450,0.622430,0.618389,0.631391
Dettes-Fonds Propres,2.224319,1.483208,1.681152,1.646516,1.619480,1.730935
Endettement Long Terme,0.583849,0.433427,0.505252,0.526649,0.516592,0.513154
Rotation des stocks,6.556541,6.766396,7.416178,7.598490,7.599722,7.187465
Rotation de l'actif,1.323005,1.548536,1.325490,1.346992,1.321245,1.373053
Marge Bénéficiaire,0.022503,0.016405,0.032563,0.021192,0.013923,0.021317
Rendement de l'actif,0.029772,0.025404,0.043162,0.028546,0.018395,0.029056
Rendement des fonds propres,0.096226,0.063207,0.115855,0.075604,0.048204,0.079819


In [67]:
BSL_analysis.to_csv (r'C:\Users\hadri\OneDrive\Documents\Cours\A20\IND8122\export_BSL.csv', index = True, header=True)

In [64]:
# get Balance Sheet 
urlBSEMP = 'https://raw.githubusercontent.com/hadrienpierre/IND8122-CS1/main/Annual_BalanceSheet_EMP.A.csv'
BSEMP_orginal= pd.read_csv (urlBSEMP, index_col=0, decimal='.', thousands = ' ')
BSEMP_transpose = BSEMP_orginal.transpose() 

# get Income Statement 
urlISEMP = 'https://raw.githubusercontent.com/hadrienpierre/IND8122-CS1/main/Annual_IncomeStatement_EMP.A.csv'
ISEMP_orginal = pd.read_csv (urlISEMP, index_col=0, decimal='.', thousands = ' ')
ISEMP_transpose = ISEMP_orginal.transpose()

# get Cash Flow
#CFEMP_orginal, CFEMP_transpose = get_table(urlEMP_cf)

#Ratio du Fond de Roulement = Actif Courant / Passif Courant

current_assets = BSEMP_transpose["Actifs courants"].str.replace(',', ' ').astype(float)
current_liabilities = BSEMP_transpose["Passifs courants"].str.replace(',', ' ').astype(float)
rfdr = current_assets / current_liabilities
BSEMP_analysis = pd.DataFrame(rfdr)
BSEMP_analysis.columns = ['Ratio Fond de Roulement']

#EMPevier Financier
# Endettement Total
capitaux_propres = BSEMP_transpose["Capitaux propres"].str.replace(',', ' ').astype(float)
total_assets = BSEMP_transpose["Total de l'actif"].str.replace(',', ' ').astype(float)
BSEMP_analysis['Endettement Total'] = (total_assets - capitaux_propres) / total_assets
BSEMP_analysis

# Dette Fonds Propres
total_liabilities = BSEMP_transpose["Total du passif"].str.replace(',', ' ').astype(float)
BSEMP_analysis['Dettes-Fonds Propres'] = total_liabilities / capitaux_propres
BSEMP_analysis

#Endettement long terme
longterm_liabilities = BSEMP_transpose["Total des passifs non courants"].str.replace(',', ' ').astype(float)
BSEMP_analysis['Endettement Long Terme'] = longterm_liabilities / (longterm_liabilities + capitaux_propres)
BSEMP_analysis

#Rotation des Stocks
cogs = ISEMP_transpose["Coût des produits"].str.replace(',', ' ').astype(float)
inventory = BSEMP_transpose["Stocks"].str.replace(',', '').astype(float)
BSEMP_analysis['Rotation des stocks'] = cogs / inventory
BSEMP_analysis

#Rotation de l'actif
revenues = ISEMP_transpose["Total des produits"].str.replace(',', ' ').astype(float)
BSEMP_analysis["Rotation de l'actif"] = revenues / total_assets
BSEMP_analysis

## Rentablité
#Marge Bénéficiaire
profits = ISEMP_transpose["Bénéfice net"].str.replace(',', ' ').astype(float)
BSEMP_analysis["Marge Bénéficiaire"] = profits / revenues
BSEMP_analysis

# Rendement de l'actif
BSEMP_analysis["Rendement de l'actif"] = profits / total_assets
BSEMP_analysis

# Rendement des fonds propres
BSEMP_analysis["Rendement des fonds propres"] = profits / capitaux_propres
BSEMP_analysis

BSEMP_analysis = BSEMP_analysis.transpose()
BSEMP_analysis["Moyenne 5 ans"] = BSEMP_analysis.mean(axis=1)
BSEMP_analysis


,avr. 20,avr. 19,avr. 18,avr. 17,avr. 16,Moyenne 5 ans
Ratio Fond de Roulement,0.801847,0.984411,0.842589,0.869836,0.957531,0.891243
Endettement Total,0.731796,0.583094,0.572524,0.580910,0.603447,0.614354
Dettes-Fonds Propres,2.705753,1.378713,1.321216,1.370068,1.505422,1.656234
Endettement Long Terme,0.625644,0.414842,0.342624,0.407090,0.430186,0.444077
Rotation des stocks,13.397945,13.218477,14.632550,13.688549,14.496388,13.886782
Rotation de l'actif,1.817015,2.618304,2.795498,2.737761,2.693965,2.532508
Marge Bénéficiaire,0.021946,0.015405,0.006587,0.006658,-0.086560,-0.007193
Rendement de l'actif,0.039876,0.040334,0.018414,0.018228,-0.233189,-0.023268
Rendement des fonds propres,0.148678,0.096745,0.043076,0.043494,-0.588041,-0.051210


In [68]:
BSEMP_analysis.to_csv (r'C:\Users\hadri\OneDrive\Documents\Cours\A20\IND8122\export_BSEMP.csv', index = True, header=True)
